In [2]:
import pybaseball
from pybaseball import statcast
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import timezone
from bs4 import BeautifulSoup
import io
import requests
import unicodedata
pybaseball.cache.enable()

from RosterScraper import RosterScraper

In [3]:
url = 'https://docs.google.com/spreadsheets/d/1JgczhD5VDQ1EiXqVG-blttZcVwbZd5_Ne_mefUGwJnk/pub?output=csv'
res = requests.get(url)
ID = pd.read_csv(io.BytesIO(res.content), sep=',')
ID.dropna(subset=['MLBID'], inplace=True)
ID['MLBID'] = ID['MLBID'].astype(int)

Rosters = RosterScraper()
BID = Rosters[Rosters["Position"] == "Batter"]
PID = Rosters[Rosters["Position"] == "Pitcher"]

In [4]:
def convert_name(name):
    if name == 'Rockies':
        return 'COL'
    elif name == 'Reds':
        return 'CIN'
    elif name == 'Mariners':
        return 'SEA'
    elif name == 'Nationals':
        return 'WSH'
    elif name == 'Yankees':
        return 'NYY'
    elif name == 'Astros':
        return 'HOU'
    elif name == 'Red Sox':
        return 'BOS'
    elif name == 'Athletics':
        return 'OAK'
    elif name == 'Mets':
        return 'NYM'
    elif name == 'Braves':
        return 'ATL'
    elif name == 'Giants':
        return 'SF'
    elif name == 'Brewers':
        return 'MIL'
    elif name == 'Rays':
        return 'TB'
    elif name == 'Royals':
        return 'KC'
    elif name == 'White Sox':
        return 'CWS'
    elif name == 'Cubs':
        return 'CHC'
    elif name == 'Angels':
        return 'LAA'
    elif name == 'Tigers':
        return 'DET'
    elif name == 'Diamondbacks':
        return 'ARI'
    elif name == 'Guardians':
        return 'CLE'
    elif name == 'Orioles':
        return 'BAL'
    elif name == 'Twins':
        return 'MIN'
    elif name == 'Marlins':
        return 'MIA'
    elif name == 'Phillies':
        return 'PHI'
    elif name == 'Rangers':
        return 'TEX'
    elif name == 'Dodgers':
        return 'LAD'
    elif name == 'Padres':
        return 'SD'
    elif name == 'Pirates':
        return 'PIT'
    elif name == 'Blue Jays':
        return 'TOR'
    elif name == 'Cardinals':
        return 'STL'
    else:
        return np.nan
    
def flip_names(name):
    first_name, last_name = name.split(", ")
    return f"{last_name} {first_name}"

def replace_special_chars(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')

In [17]:
def getDKData2024():
    
    eastern_time = datetime.datetime.now(timezone.utc).astimezone(timezone(datetime.timedelta(hours=-5)))
    todaysdate = eastern_time.strftime("%m-%d-%Y")
    url = 'https://rotogrinders.com/lineups/mlb?site=draftkings'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')

    gamelist = []
    gamecards = soup.findAll("div", {"class": "game-card-teams"})
    for x in gamecards:
        twoteams = x.findAll("span", {"class": "team-nameplate-mascot"})
        roadteam = convert_name(twoteams[0].text)
        hometeam = convert_name(twoteams[1].text)
        gamekey = "{}@{}".format(roadteam,hometeam)
        gamelist.append(gamekey)

    matchupsdf = pd.DataFrame()
    for game in gamelist:
        roadteam = game.split("@")[0]
        hometeam = game.split("@")[1]
        thisdf1 = pd.DataFrame({"Team": roadteam, "Opp": hometeam, "RoadTeam": roadteam, "HomeTeam": hometeam},index=[0])
        thisdf2 = pd.DataFrame({"Team": hometeam, "Opp": roadteam, "RoadTeam": roadteam, "HomeTeam": hometeam},index=[0])
        matchupsdf = pd.concat([matchupsdf,thisdf1,thisdf2])
        
    oppdict = dict(zip(matchupsdf.Team,matchupsdf.Opp))
    hometeamdict = dict(zip(matchupsdf.Team,matchupsdf.HomeTeam))
    roadteamdict = dict(zip(matchupsdf.Team,matchupsdf.RoadTeam))

    disabled_span_list = []
    for span in soup.findAll("span", {"class": "player-nameplate disabled"}):
        for a in span.findAll("a"):
            disabled_span_list.append(a.text)

    spdata = pd.DataFrame()
    for div in soup.findAll("span", {"class": "player-nameplate", "data-position": "SP"}):
        if "TBD" in str(div):
            playername = "TBD"
            pos = "SP"
            sal = 0
        else:
            for a in div.findAll('a', {'class': 'player-nameplate-name'}):
                playername = a.text.strip()

            strdiv = str(div)
            pos = strdiv[strdiv.find("data-position")+15:strdiv.find("data-salary")-2]
            sal = strdiv[strdiv.find("data-salary")+13:strdiv.find("<div class = 'player-nameplate-info'>")-3]
        try:
            ownership = strdiv[strdiv.find('<span class="small muted" data-auth="502">') + 42:strdiv.find('%')]
            ownership = ownership.replace("</span>", "")
            ownership = ownership.replace("</span", "")
            ownership = ownership.replace("</div>", "")
            ownership = ownership.replace(" ", "")
        except:
            ownership = np.nan

        thisspdata = pd.DataFrame([[playername, sal, ownership]], columns = ["Player", "Salary", "Ownership"])
        spdata = pd.concat([spdata, thisspdata])

    spdata2 = pd.merge(spdata, PID[["Name", "Team"]], left_on = ["Player"], right_on = ["Name"], how = "left").rename(columns = {"Team": "PitcherTeam"})
    spdata3 = pd.merge(spdata2, matchupsdf[["Team", "Opp"]], left_on = ["PitcherTeam"], right_on = ["Team"], how = "left").drop(columns = ["Team"])

    opp_spname_dict = dict(zip(spdata3.Opp, spdata3.Player))
    opp_spsal_dict = dict(zip(spdata3.Opp, spdata.Salary))
    opp_spown_dict = dict(zip(spdata3.Opp, spdata3.Ownership))

    ludf = pd.DataFrame()
    
    for li in soup.findAll("li", {"class": "lineup-card-player"}):
        for a in li.findAll("a", {"class": ["player-nameplate-name", "player-nameplate disabled"]}):
            playername = a.text

        listring = str(li)
        for span in li.find("span", {"class": "small"}):
            luspot = span.text
            luspot = luspot.replace("\n", "")
            luspot = luspot.strip()
            luspot = int(luspot)
        pos = listring[listring.find("data-position")+15:listring.find("data-salary")-2]
        sal = listring[listring.find("data-salary")+13:listring.find("<span class='small'>")-3]
        ownership = ownership.replace("</span>", "")
        ownership = ownership.replace("</span", "")
        ownership = ownership.replace("</li", "")
        ownership = ownership.replace("</div>", "")
        ownership = ownership.replace(" ", "")

        try:
            sal = int(sal)
        except:
            sal = 0
        thisludf = pd.DataFrame([[playername, luspot, sal, ownership]], columns = ["Player", "Spot", "Sal", "Ownership"])
        ludf = pd.concat([ludf, thisludf])

    ludf2 = pd.merge(ludf, BID[["Name", "Team"]], left_on = ["Player"], right_on = ["Name"], how = "left").rename(columns = {"Team": "BatterTeam"})
    ludf2['BatterTeam'] = ludf2['BatterTeam'].fillna(method='ffill')

    ludf2_teamlist = list(ludf2["BatterTeam"])

    dhteams = []
    for x in ludf2_teamlist:
        if ludf2_teamlist.count(x) > 11:
            if x in dhteams:
                pass
            else:
                dhteams.append(x)

    extract_dh = ludf2[ludf2["BatterTeam"].isin(dhteams)]
    new_ludf2 = ludf2[~ludf2["BatterTeam"].isin(dhteams)]

    new_team_list = []
    runcounter = 0

    for x in list(extract_dh["BatterTeam"].astype(str)):
        if runcounter < 18:
            new_team_list.append(x)
            runcounter += 1
        else:
            new_team_list.append(x+"2")
            runcounter += 1

    extract_dh["BatterTeam"] = new_team_list

    ludf2 = pd.concat([extract_dh, new_ludf2])
    ludf2["Opp"] = ludf2["BatterTeam"].map(oppdict)
    ludf2["HomeTeam"] = ludf2["BatterTeam"].map(hometeamdict)
    ludf2["RoadTeam"] = ludf2["BatterTeam"].map(roadteamdict)
    ludf2['SP'] = ludf2['BatterTeam'].map(opp_spname_dict)
    ludf2['SPSal'] = ludf2['BatterTeam'].map(opp_spsal_dict)
    ludf2['SPOwnership'] = ludf2['BatterTeam'].map(opp_spown_dict)
    ludf2['Date'] = todaysdate
    ludf2['Time'] = np.nan

    ludf3 = ludf2[['BatterTeam','RoadTeam','HomeTeam','Time','Spot','Player','Sal','Ownership','Date', "SP"]]

    dkdata = ludf3.copy()

    try:
        checknan = dkdata[["BatterTeam", "SP"]]
        getnans = checknan[["SP"].isna()]
        if len(getnans) == 0:
            nonans = 1
            nanmapdict = {}
        else:
            nonans = 0
            getnans["SP"] = disabled_span_list
            nanmapdict = dict(zip(getnans.Team, getnans.SP))
    except:
        pass

    try:
        dkdata["SP"] = np.where(dkdata["SP"].isna(), dkdata["BatterTeam"].map(nanmapdict), dkdata["SP"])
    except:
        pass
    
    for i in range(1, len(dkdata) - 1):
        if dkdata.loc[i, 'BatterTeam'] != dkdata.loc[i-1, 'BatterTeam']:
            if dkdata.loc[i, 'BatterTeam'] != dkdata.loc[i+1, 'BatterTeam']:
                dkdata.loc[i, 'BatterTeam'] = np.nan
                dkdata.loc[i, 'HomeTeam'] = np.nan
                dkdata.loc[i, 'RoadTeam'] = np.nan
                dkdata.loc[i, 'SP'] = np.nan

    
    dkdata[["BatterTeam", "RoadTeam", "HomeTeam"]] = dkdata[["BatterTeam", "RoadTeam", "HomeTeam"]].fillna(method='ffill')
    dkdata = dkdata.drop_duplicates(subset = ["BatterTeam", "SP"], keep = "first")
    dkdata = dkdata.drop(columns = ["Time", "Sal", "Ownership"])

    dkdata['BatterTeam'] = dkdata['BatterTeam'].replace('ARI', 'AZ')
    dkdata['RoadTeam'] = dkdata['RoadTeam'].replace('ARI', 'AZ')
    dkdata['HomeTeam'] = dkdata['HomeTeam'].replace('ARI', 'AZ')

    dkdata['Date'] = pd.to_datetime(dkdata['Date'])
    dkdata['Date'] = dkdata['Date'].dt.strftime('%Y-%m-%d')
    dkdata = dkdata.set_index("Date")
    dkdata = dkdata[["BatterTeam", "RoadTeam", "HomeTeam", "SP"]]

    return(dkdata)

In [6]:
#statcast(start_dt = "2022-04-07", end_dt = "2022-10-05")
#statcast(start_dt = "2023-03-30", end_dt = "2023-10-01")
savant2022 = pd.read_csv("~/Desktop/Random-Projects/MLB/savant2022.csv")
savant2023 = pd.read_csv("~/Desktop/Random-Projects/MLB/savant2023.csv")

In [7]:
#pd.set_option('display.max_columns', None)
combined1 = pd.concat([savant2022, savant2023])
combined1['game_date'] = pd.to_datetime(combined1['game_date'])
combined1['game_date'] = combined1['game_date'].dt.strftime('%Y-%m-%d')
combined1['BatterTeam'] = np.where(combined1['inning_topbot'] == 'Top', combined1['away_team'], combined1['home_team'])
combined1['PitcherTeam'] = np.where(combined1['inning_topbot'] == 'Top', combined1['home_team'], combined1['away_team'])
combined1["player_name"] = combined1["player_name"].apply(flip_names)
combined1["player_name"] = combined1["player_name"].apply(replace_special_chars)

In [8]:
# Group by game and team identifiers
groupby_cols = ['game_date', 'BatterTeam', 'away_team', 'home_team']

# Function to keep only the starter's data
def keep_starter(group):
    starter_name = group['player_name'].iloc[0]
    return group[group['player_name'] == starter_name]

def count_outs(x):
    single_outs = ['other_out', 'strikeout', 'field_out', 'fielders_choice', 'fielders_choice_out']
    double_outs = ['double_play', 'strikeout_double_play', 'grounded_into_double_play']
    triple_outs = ['triple_play']
    
    outs = (x.isin(single_outs)).sum() + 2 * (x.isin(double_outs)).sum() + 3 * (x.isin(triple_outs)).sum()
    return outs

# Apply the function to each group
combined2 = combined1[["game_date", "home_team", "away_team", "inning", "inning_topbot", "at_bat_number", "pitch_number", "BatterTeam", "MLBNAME", "events", "description", "bb_type", "estimated_woba_using_speedangle", "woba_value", "p_throws", "PitcherTeam", "player_name", "delta_run_exp"]].sort_values(by = ["game_date", "home_team", "away_team", "inning_topbot", "at_bat_number", "pitch_number"], ascending=[True, True, True, False, True, True])
df_starters_only = combined2.groupby(groupby_cols).apply(keep_starter).reset_index(drop = True)

/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_58850/2263374342.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_starters_only = combined2.groupby(groupby_cols).apply(keep_starter).reset_index(drop = True)


In [9]:
Train1 = df_starters_only.groupby(["game_date", "BatterTeam", "away_team", "home_team", "inning", "at_bat_number", "MLBNAME", "player_name", "p_throws"]).agg(
    Pitches = ("pitch_number", "size"),
    PA = ('events', lambda x: (x.isin(['other_out', 'single', 'double', 'triple', 'home_run', 'strikeout', 'field_out', 'field_error', 'fielders_choice', 'double_play', 'fielders_choice_out', 'strikeout_double_play', 'triple_play', 'grounded_into_double_play'])).sum()),
    Outs = ('events', count_outs),
    HBP = ('events', lambda x: (x == 'hit_by_pitch').sum()),
    Balls = ('description', lambda x: (x.isin(["ball", "hit_by_pitch", "blocked_ball"])).sum()),
    BB = ('events', lambda x: (x == 'walk').sum()),
    CS = ('description', lambda x: (x == 'called_strike').sum()),
    Whiff = ('description', lambda x: (x.isin(["swinging_strike", "swinging_strike_blocked", "foul_tip"])).sum()),
    Strikes = ('description', lambda x: (x.isin(["called_strike", "swinging_strike", "foul", "swinging_strike_blocked", "foul_tip"])).sum()),
    K = ('events', lambda x: (x == 'strikeout').sum()),
    xwOBA = ("estimated_woba_using_speedangle", "mean"),
    wOBA = ("woba_value", "mean"),
    RunExp = ("delta_run_exp", "mean")).reset_index().fillna(0)

In [46]:
Train2 = Train1.groupby(["game_date", "BatterTeam", "away_team", "home_team", "player_name", "p_throws"]).agg(
    Pitches = ("Pitches", "sum"),
    AB = ("at_bat_number", "size"),
    PA = ("PA", "sum"),
    Outs = ("Outs", "sum"),
    HBP = ("HBP", "sum"),
    Balls = ("Balls", "sum"),
    BB = ("BB", "sum"),
    CS = ("CS", "sum"),
    Whiff = ("Whiff", "sum"),
    Strikes = ("Strikes", "sum"),
    K = ("K", "sum"),
    xwOBA = ("wOBA", "mean"),
    wOBA = ("wOBA", "mean"),
    RunExp = ("RunExp", "mean")).reset_index().fillna(0)

Train2["P/AB"] = round((Train2['Pitches'] / Train2['AB']), 2)
Train2['K%'] = round((Train2['K'] / Train2['AB']) * 100, 2)
Train2['BB%'] = round((Train2['BB'] / Train2['AB']) * 100, 2)
Train2['K-BB%'] = Train2["K%"] - Train2["BB%"]
Train2['Ball%'] = round((Train2['Balls'] / Train2['Pitches']) * 100, 2)
Train2['Strike%'] = round((Train2['Strikes'] / Train2['Pitches']) * 100, 2)
Train2['CS%'] = round((Train2['CS'] / Train2['Pitches']) * 100, 2)
Train2['Whiff%'] = round((Train2['Whiff'] / Train2['Pitches']) * 100, 2)
Train2["CSW"] = Train2["CS"] + Train2["Whiff"]
Train2['CSW%'] = round((Train2['CSW'] / Train2['Pitches']) * 100, 2)
Train2 = Train2.drop(columns = ["CSW"])

In [47]:
Train3 = Train2.rename(columns={'away_team': 'RoadTeam', 'home_team': 'HomeTeam', "player_name": "SP"})
Train3 = Train3.drop(columns = "game_date")
Train3

,BatterTeam,RoadTeam,HomeTeam,SP,p_throws,Pitches,AB,PA,Outs,HBP,...,RunExp,P/AB,K%,BB%,K-BB%,Ball%,Strike%,CS%,Whiff%,CSW%
0,ATL,CIN,ATL,Tyler Mahle,R,84,20,18,15,0,...,-0.028863,4.20,35.00,10.00,25.00,39.29,46.43,19.05,15.48,34.52
1,AZ,SD,AZ,Yu Darvish,R,92,22,17,17,0,...,-0.040531,4.18,13.64,18.18,-4.54,44.57,39.13,17.39,10.87,28.26
2,CHC,MIL,CHC,Corbin Burnes,R,83,22,16,12,0,...,-0.020852,3.77,18.18,13.64,4.54,42.17,40.96,18.07,13.25,31.33
3,CIN,CIN,ATL,Max Fried,L,84,26,23,16,1,...,0.016518,3.23,19.23,3.85,15.38,33.33,44.05,16.67,11.90,28.57
4,CLE,CLE,KC,Zack Greinke,R,84,22,20,16,0,...,-0.038159,3.82,4.55,4.55,0.00,33.33,42.86,22.62,5.95,28.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9509,STL,CIN,STL,Hunter Greene,R,45,9,8,5,1,...,0.016376,5.00,33.33,0.00,33.33,42.22,46.67,13.33,11.11,24.44
9510,TB,TB,TOR,Wes Parsons,R,60,15,12,7,1,...,-0.021352,4.00,13.33,13.33,0.00,41.67,41.67,20.00,8.33,28.33
9511,TEX,TEX,SEA,George Kirby,R,68,18,18,17,0,...,-0.062775,3.78,33.33,0.00,33.33,30.88,51.47,16.18,14.71,30.88
9512,TOR,TB,TOR,Jacob Lopez,L,65,21,19,14,1,...,0.109993,3.10,14.29,4.76,9.53,35.38,40.00,12.31,7.69,20.00


In [18]:
TodaysData = getDKData2024()
TodaysData

/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_58850/3305814355.py:95: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  ludf2['BatterTeam'] = ludf2['BatterTeam'].fillna(method='ffill')
/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_58850/3305814355.py:164: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dkdata[["BatterTeam", "RoadTeam", "HomeTeam"]] = dkdata[["BatterTeam", "RoadTeam", "HomeTeam"]].fillna(method='ffill')


,BatterTeam,RoadTeam,HomeTeam,SP
Date,,,,
2024-07-09,CHC,CHC,BAL,Dean Kremer
2024-07-09,BAL,CHC,BAL,Jameson Taillon
2024-07-09,CLE,CLE,DET,Kenta Maeda
2024-07-09,DET,CLE,DET,Ben Lively
2024-07-09,LAD,LAD,PHI,Zack Wheeler
2024-07-09,PHI,LAD,PHI,Bobby Miller
2024-07-09,NYY,NYY,TB,Ryan Pepiot
2024-07-09,TB,NYY,TB,Carlos Rodon
2024-07-09,OAK,OAK,BOS,Brayan Bello


In [57]:
eastern_time = datetime.datetime.now(timezone.utc).astimezone(timezone(datetime.timedelta(hours=-5)))
savant2024 = statcast(start_dt = "2024-03-28", end_dt = eastern_time.strftime("%Y-%m-%d"))
savant2024['game_date'] = pd.to_datetime(savant2024['game_date'])
savant2024['game_date'] = savant2024['game_date'].dt.strftime('%Y-%m-%d')
savant2024['BatterTeam'] = np.where(savant2024['inning_topbot'] == 'Top', savant2024['away_team'], savant2024['home_team'])
savant2024['PitcherTeam'] = np.where(savant2024['inning_topbot'] == 'Top', savant2024['home_team'], savant2024['away_team'])
savant2024 = pd.merge(savant2024, ID[["MLBID", "MLBNAME"]], left_on = 'batter', right_on = 'MLBID', how = 'left')
savant2024.dropna(subset=['MLBNAME'], inplace=True)
savant2024 = savant2024.drop_duplicates(subset = ["pitch_type", "game_date", "release_speed", "release_pos_x", "release_pos_z", "player_name"], keep='first')
savant2024["player_name"] = savant2024["player_name"].apply(flip_names)
savant2024 = savant2024[["game_date", "home_team", "away_team", "inning", "inning_topbot", "at_bat_number", "pitch_number", "pitch_type", "BatterTeam", "MLBNAME", "balls", "strikes", "outs_when_up", "events", "description", "bb_type", "hit_distance_sc", "launch_speed", "launch_angle", "estimated_ba_using_speedangle", "estimated_woba_using_speedangle", "woba_value", "p_throws", "PitcherTeam", "player_name", "delta_home_win_exp", "delta_run_exp", "away_score", "home_score"]].sort_values(by = ["game_date", "home_team", "away_team", "inning_topbot", "at_bat_number", "pitch_number"], ascending=[True, True, True, False, True, True])
savant2024 = savant2024.groupby(groupby_cols).apply(keep_starter).reset_index(drop = True)
savant2024["player_name"] = savant2024["player_name"].apply(replace_special_chars)

This is a large query, it may take a moment to complete


100%|██████████| 104/104 [01:37<00:00,  1.07it/s]
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pybaseball/statcast.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat(dataframe_list, axis=0).convert_dtypes(convert_string=False)
/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_58850/2056470012.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  savant2024 = savant2024.groupby(groupby

In [58]:
Season1 = savant2024.groupby(["game_date", "BatterTeam", "away_team", "home_team", "inning", "at_bat_number", "MLBNAME", "player_name", "p_throws"]).agg(
    Pitches = ("pitch_number", "size"),
    PA = ('events', lambda x: (x.isin(['other_out', 'single', 'double', 'triple', 'home_run', 'strikeout', 'field_out', 'field_error', 'fielders_choice', 'double_play', 'fielders_choice_out', 'strikeout_double_play', 'triple_play', 'grounded_into_double_play'])).sum()),
    Outs = ('events', count_outs),
    HBP = ('events', lambda x: (x == 'hit_by_pitch').sum()),
    Balls = ('description', lambda x: (x.isin(["ball", "hit_by_pitch", "blocked_ball"])).sum()),
    BB = ('events', lambda x: (x == 'walk').sum()),
    CS = ('description', lambda x: (x == 'called_strike').sum()),
    Whiff = ('description', lambda x: (x.isin(["swinging_strike", "swinging_strike_blocked", "foul_tip"])).sum()),
    Strikes = ('description', lambda x: (x.isin(["called_strike", "swinging_strike", "foul", "swinging_strike_blocked", "foul_tip"])).sum()),
    K = ('events', lambda x: (x == 'strikeout').sum()),
    xwOBA = ("estimated_woba_using_speedangle", "mean"),
    wOBA = ("woba_value", "mean"),
    RunExp = ("delta_run_exp", "mean")).reset_index().fillna(0)

In [59]:
Season2 = Season1.groupby(["game_date", "BatterTeam", "away_team", "home_team", "player_name", "p_throws"]).agg(
    Pitches = ("Pitches", "sum"),
    AB = ("at_bat_number", "size"),
    PA = ("PA", "sum"),
    Outs = ("Outs", "sum"),
    HBP = ("HBP", "sum"),
    Balls = ("Balls", "sum"),
    BB = ("BB", "sum"),
    CS = ("CS", "sum"),
    Whiff = ("Whiff", "sum"),
    Strikes = ("Strikes", "sum"),
    K = ("K", "sum"),
    xwOBA = ("wOBA", "mean"),
    wOBA = ("wOBA", "mean"),
    RunExp = ("RunExp", "mean")).reset_index().fillna(0)

Season2

,game_date,BatterTeam,away_team,home_team,player_name,p_throws,Pitches,AB,PA,Outs,HBP,Balls,BB,CS,Whiff,Strikes,K,xwOBA,wOBA,RunExp
0,2024-03-28,AZ,COL,AZ,Kyle Freeland,L,49,17,16,7,0,15,1,8,6,20,2,0.676471,0.676471,0.272008
1,2024-03-28,BAL,LAA,BAL,Patrick Sandoval,L,60,14,10,3,0,24,2,10,6,26,2,0.575,0.575,0.035
2,2024-03-28,BOS,BOS,SEA,Luis Castillo,R,91,24,19,13,0,33,2,17,6,42,5,0.395833,0.395833,0.036904
3,2024-03-28,CHC,CHC,TEX,Nathan Eovaldi,R,88,22,19,17,1,33,1,11,6,38,3,0.290909,0.290909,0.013162
4,2024-03-28,CIN,WSH,CIN,Josiah Gray,R,80,22,20,12,0,28,2,14,15,37,6,0.538636,0.538636,0.024324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2685,2024-07-08,NYM,NYM,PIT,Mitch Keller,R,107,31,30,24,1,28,0,27,9,55,6,0.26129,0.26129,-0.04215
2686,2024-07-08,PIT,NYM,PIT,Christian Scott,R,77,19,18,17,0,24,1,9,8,38,3,0.142105,0.142105,-0.043217
2687,2024-07-08,STL,STL,WSH,Mitchell Parker,L,90,28,25,19,0,27,1,14,11,42,6,0.257143,0.257143,-0.054911
2688,2024-07-08,TEX,TEX,LAA,Davis Daniel,R,61,19,18,11,1,18,0,15,5,28,3,0.492105,0.492105,0.11526


In [60]:
Season3 = Season2.groupby(["player_name", "p_throws"]).agg(
    Games = ("Pitches", "size"),
    Pitches = ("Pitches", "sum"),
    AB = ("AB", "sum"),
    PA = ("PA", "sum"),
    Outs = ("Outs", "sum"),
    HBP = ("HBP", "sum"),
    Balls = ("Balls", "sum"),
    BB = ("BB", "sum"),
    CS = ("CS", "sum"),
    Whiff = ("Whiff", "sum"),
    Strikes = ("Strikes", "sum"),
    K = ("K", "sum"),
    xwOBA = ("wOBA", "mean"),
    wOBA = ("wOBA", "mean"),
    RunExp = ("RunExp", "mean")).reset_index().fillna(0)

Season3["P/AB"] = round(Season3["Pitches"] / Season3["AB"], 2)
Season3['K%'] = round((Season3['K'] / Season3['AB']) * 100, 2)
Season3['BB%'] = round((Season3['BB'] / Season3['AB']) * 100, 2)
Season3['K-BB%'] = Season3["K%"] - Season3["BB%"]
Season3['Ball%'] = round((Season3['Balls'] / Season3['Pitches']) * 100, 2)
Season3['Strike%'] = round((Season3['Strikes'] / Season3['Pitches']) * 100, 2)
Season3['CS%'] = round((Season3['CS'] / Season3['Pitches']) * 100, 2)
Season3['Whiff%'] = round((Season3['Whiff'] / Season3['Pitches']) * 100, 2)
Season3["CSW"] = Season3["CS"] + Season3["Whiff"]
Season3['CSW%'] = round((Season3['CSW'] / Season3['Pitches']) * 100, 2)
Season3 = Season3.drop(columns = ["CSW"])

In [61]:
# List of columns to keep unchanged
columns_to_keep = ["P/AB", "xwOBA", "wOBA", "RunExp", "K%", "BB%", "K-BB%", "Ball%", "Strike%", "CS%", "Whiff%", "CSW%"]

# List of columns to divide
columns_to_divide = ["Pitches", "AB", "PA", "Outs", "HBP", "Balls", "BB", "CS", "Whiff", "Strikes", "K"]
divide_by_column = 'Games'

# Create the new DataFrame
Season4 = Season3.copy()

for col in columns_to_keep:
    Season4[col] = Season3[col]

for col in columns_to_divide:
    Season4[col] = Season3[col] / Season3[divide_by_column]

In [62]:
TodaysData.dropna(subset=['SP'], inplace=True)
TodaysData1 = pd.merge(TodaysData, Season4[['player_name', "p_throws", 'Pitches', 'AB', 'PA', 'Outs', 'HBP', 'Balls', 'BB', 'CS', 'Whiff', 'Strikes', 'K', 'xwOBA', 'wOBA', 'RunExp', 'P/AB', 'K%', 'BB%', 'K-BB%', 'Ball%', 'Strike%', 'CS%', 'Whiff%', 'CSW%']], left_on = ['SP'], right_on = ['player_name'], how = 'left').drop(columns = ["player_name"])

# If no 2024 savant data exists then gives them the league averages from 2022-23
TrainMeans = Train3.drop(['BatterTeam', 'RoadTeam', "HomeTeam", "SP", "p_throws"], axis=1).mean()
TodaysData1 = TodaysData1.fillna(TrainMeans)

In [63]:
TodaysData1

,BatterTeam,RoadTeam,HomeTeam,SP,p_throws,Pitches,AB,PA,Outs,HBP,...,RunExp,P/AB,K%,BB%,K-BB%,Ball%,Strike%,CS%,Whiff%,CSW%
0,CHC,CHC,BAL,Dean Kremer,R,90.8,22.6,20.1,16.2,0.0,...,-0.00455,4.02,25.22,8.85,16.37,35.9,47.47,19.16,11.34,30.51
1,BAL,CHC,BAL,Jameson Taillon,R,87.857143,23.928571,22.0,16.571429,0.0,...,-0.012026,3.67,19.1,5.07,14.03,33.33,45.93,18.13,9.35,27.48
2,CLE,CLE,DET,Kenta Maeda,R,74.642857,18.142857,16.071429,11.785714,0.0,...,0.038737,4.11,15.75,8.66,7.09,38.85,43.06,16.08,10.43,26.51
3,DET,CLE,DET,Ben Lively,R,85.357143,22.285714,20.0,15.785714,0.142857,...,-0.01358,3.83,21.47,7.05,14.42,34.31,47.11,18.16,9.37,27.53
4,LAD,LAD,PHI,Zack Wheeler,R,96.777778,24.277778,21.388889,17.388889,0.333333,...,-0.002613,3.99,26.09,7.55,18.54,34.85,48.85,14.41,13.66,28.07
5,PHI,LAD,PHI,Bobby Miller,R,75.5,18.166667,15.5,12.333333,0.0,...,0.016651,4.16,22.94,13.76,9.18,40.18,44.59,13.91,11.04,24.94
6,NYY,NYY,TB,Ryan Pepiot,R,83.333333,20.466667,17.933333,14.533333,0.2,...,0.005799,4.07,27.04,9.12,17.92,36.8,47.68,16.56,14.0,30.56
7,TB,NYY,TB,Carlos Rodon,L,94.611111,23.222222,20.666667,15.722222,0.5,...,0.008791,4.07,23.44,6.7,16.74,36.11,47.15,14.45,13.56,28.01
8,OAK,OAK,BOS,Brayan Bello,R,88.533333,23.066667,20.133333,15.0,0.4,...,0.023834,3.84,20.52,8.67,11.85,37.12,44.73,17.55,11.22,28.77
9,BOS,OAK,BOS,Joey Estes,R,84.666667,23.111111,21.222222,17.0,0.222222,...,0.012265,3.66,19.71,5.29,14.42,29.27,50.66,14.04,12.47,26.51


In [84]:
# Ensure Train5 and TodaysData2 are copies of Train4 and TodaysData1 respectively
Train5 = Train3.copy()
TodaysData2 = TodaysData1.copy()

# Dictionary to store the label encoders
label_encoders = {}

# Encode non-numeric columns in Train4
non_numeric_columns_train = Train5.select_dtypes(exclude=['float64', 'int64']).columns
for col in non_numeric_columns_train:
    label_encoder = LabelEncoder()
    Train5[col] = label_encoder.fit_transform(Train5[col])
    label_encoders[col] = label_encoder

# Ensure all non-numeric columns in Train4 are in TodaysData1
for col in non_numeric_columns_train:
    if col not in TodaysData2.columns:
        print(f"Warning: Column {col} from training data is not present in today's data.")
        # Adding the missing column with a default value
        TodaysData2[col] = 536

# Encode non-numeric columns in TodaysData1 using the same encoders
non_numeric_columns_today = TodaysData2.select_dtypes(exclude=['float64', 'int64']).columns
for col in non_numeric_columns_today:
    if col in label_encoders:
        label_encoder = label_encoders[col]
        unique_values = set(label_encoder.classes_)
        encoded_values = []
        for item in TodaysData2[col]:
            if item in unique_values:
                encoded_values.append(label_encoder.transform([item])[0])
            else:
                encoded_values.append(536)  # Using 536 as a placeholder for unknown categories
        TodaysData2[col] = encoded_values
    else:
        print(f"Warning: Column {col} is not present in the training data.")
        # Fit a new label encoder for columns not present in Train4, but be cautious with this
        label_encoder = LabelEncoder()
        TodaysData2[col] = label_encoder.fit_transform(TodaysData2[col])
        label_encoders[col] = label_encoder

In [85]:
TrainFeatures = Train5.drop(columns = ["K"]).values.reshape(-1, 27)
TrainLabel = Train5["K"].values.reshape(-1, 1)
TodayFeatures = TodaysData2.drop(columns = ["K"]).values.reshape(-1, 27)

rf_regressor = RandomForestRegressor(n_estimators = 152, max_depth = 15, min_samples_leaf = 4)
rf_regressor.fit(TrainFeatures, TrainLabel)
RFpred = rf_regressor.predict(TodayFeatures)

TodaysData2["RFPred"] = RFpred

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [86]:
for col in non_numeric_columns_today:
    if col in label_encoders:
        label_encoder = label_encoders[col]
        # Handling default value of 536
        TodaysData2[col] = TodaysData2[col].apply(lambda x: label_encoder.inverse_transform([x])[0] if x != 536 else np.nan)

TodaysData2["SP"].fillna(TodaysData1["SP"], inplace = True)

/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_58850/1190844444.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  TodaysData2["SP"].fillna(TodaysData1["SP"], inplace = True)


In [89]:
TodaysData2.sort_values("RFPred", ascending = True)

,BatterTeam,RoadTeam,HomeTeam,SP,p_throws,Pitches,AB,PA,Outs,HBP,...,P/AB,K%,BB%,K-BB%,Ball%,Strike%,CS%,Whiff%,CSW%,RFPred
21,MIL,PIT,MIL,Josh Fleming,L,45.5,12.5,11.5,9.0,0.0,...,3.64,12.0,8.0,4.0,37.36,39.56,14.29,7.69,21.98,2.000000
22,TEX,TEX,LAA,Roansy Contreras,R,50.5,12.5,10.0,6.5,0.0,...,4.04,12.0,12.0,0.0,38.61,41.58,16.83,5.94,22.77,2.000000
2,CLE,CLE,DET,Kenta Maeda,R,74.642857,18.142857,16.071429,11.785714,0.0,...,4.11,15.75,8.66,7.09,38.85,43.06,16.08,10.43,26.51,2.967105
26,SEA,SEA,SD,Adam Mazur,R,76.5,20.333333,16.833333,12.333333,0.0,...,3.76,12.3,13.93,-1.63,39.22,40.96,16.56,7.84,24.4,3.000000
14,KC,KC,STL,Andre Pallante,R,71.857143,18.714286,16.428571,13.428571,0.285714,...,3.84,19.08,9.92,9.16,39.36,42.35,16.3,8.35,24.65,3.118421
16,MIN,MIN,CWS,Drew Thorpe,R,86.0,22.2,18.6,15.2,0.0,...,3.87,16.22,12.61,3.61,41.86,39.77,14.65,10.23,24.88,3.914474
23,LAA,TEX,LAA,Max Scherzer,R,74.666667,21.0,19.666667,16.0,0.0,...,3.56,17.46,4.76,12.7,31.7,46.43,14.73,12.5,27.23,3.967105
12,WSH,WSH,NYM,Jose Quintana,L,87.0,22.3125,18.9375,14.625,0.4375,...,3.9,17.65,8.68,8.97,40.09,41.24,16.88,8.55,25.43,3.967105
20,PIT,PIT,MIL,Colin Rea,R,86.4,23.333333,21.133333,16.333333,0.266667,...,3.7,17.43,6.57,10.86,34.57,45.22,19.21,8.56,27.78,4.000000
11,CIN,COL,CIN,Cal Quantrill,R,94.647059,23.235294,20.235294,15.294118,0.352941,...,4.07,17.47,8.35,9.12,37.48,44.69,15.91,10.13,26.04,4.000000
